In [69]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline

1. Загрузите датасет. Описание датасета можно посмотреть [здесь](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/).

2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

5. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.


In [15]:
data = pd.read_csv('SMSSpamCollection.txt', sep='\t', names=['classes', 'text'])

In [16]:
data.head()

,classes,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [45]:
texts = list(data.text.values)
labels = [1 if cls == 'spam' else 0 for cls in data.classes]

In [47]:
X = feature_extraction.text.CountVectorizer(texts)

In [57]:
cross_val_score(Pipeline([('vectorizer', X), 
                          ('classifier', LogisticRegression(random_state=2))]), 
                texts, 
                labels,
                cv=10, 
                scoring='f1').mean()

0.9326402983610631

6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

In [58]:
clf_pipeline = Pipeline(
            [("vectorizer", X),
            ("classifier", LogisticRegression(random_state=2))]
        )


clf_pipeline.fit(texts, labels)

print(clf_pipeline)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8',
        input=['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u...nalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])


In [59]:
print(clf_pipeline.predict(["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
                           "FreeMsg: Txt: claim your reward of 3 hours talk time",
                           "Have you visited the last lecture on physics?",
                           "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
                           "Only 99$"]))

[1 1 0 0 0]


7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [68]:
%%time
ngram_ranges = {'bigram':(2, 2), 'trigram':(3, 3), 'unigrams, bigrams and trigrams':(1, 3)}
for name, n in ngram_ranges.items():
    print(name)
    print(cross_val_score(Pipeline([('vectorizer', feature_extraction.text.CountVectorizer(texts, ngram_range=n)), 
                                    ('classifier', LogisticRegression(random_state=2))]), 
                texts, 
                labels,
                cv=10, 
                scoring='f1').mean())
    print()

bigram
0.8224220664187133

trigram
0.7250161555467377

unigrams, bigrams and trigrams
0.9251382558648837

Wall time: 11.5 s


8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [71]:
%%time
ngram_ranges = {'bigram':(2, 2), 'trigram':(3, 3), 'unigrams, bigrams and trigrams':(1, 3)}
for name, n in ngram_ranges.items():
    print(name)
    print(cross_val_score(Pipeline([('vectorizer', feature_extraction.text.CountVectorizer(texts, ngram_range=n)), 
                                    ('classifier', MultinomialNB())]), 
                texts, 
                labels,
                cv=10, 
                scoring='f1').mean())
    print()

bigram
0.9330215115853413

trigram
0.871265305963816

unigrams, bigrams and trigrams
0.9472991994136064

Wall time: 9.69 s


In [76]:
%%time
ngram_ranges = {'bigram':(2, 2), 'trigram':(3, 3), 'unigrams, bigrams and trigrams':(1, 3)}
for name, n in ngram_ranges.items():
    X = feature_extraction.text.CountVectorizer(ngram_range=n).fit_transform(texts)
    print(name)
    print(cross_val_score(MultinomialNB(), X, labels, cv=10, scoring='f1').mean())
    print()

bigram
0.6455015177985443

trigram
0.37871948524573595

unigrams, bigrams and trigrams
0.8884859656061002

Wall time: 1.5 s


Попробуйте использовать в логистической регрессии в качестве признаков Tfidf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [78]:
cross_val_score(Pipeline([('vectorizer', feature_extraction.text.TfidfVectorizer()), 
                          ('classifier', LogisticRegression(random_state=2))]), 
                texts, 
                labels,
                cv=10, 
                scoring='f1').mean()

0.8785100455343396